# Statistiques sur les triplets BASE, PST, PART

## Import du corpus Brown
on utilise nltk :  
- pour importer Brown
- pour avoir les tags
- pour lemmatiser

In [3]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
list_cat = brown.categories()
#brown manual: https://web.archive.org/web/20081225080543/http://khnt.hit.uib.no/icame/manuals/brown/INDEX.HTM

[nltk_data] Downloading package brown to /Users/gilles/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [4]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /Users/gilles/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Il y a un pb avec "saw" qui est lemmatisé en "saw" du verbe scier plutôt que "see" du verbe voir. On peut anticiper des problèmes avec d'autres homographes entre verbes...

In [5]:
print(lemmatizer.lemmatize("seen",pos="v"))
print(lemmatizer.lemmatize("saw",pos="v"))

see
saw


On utilise pandas pour
- faire des tableaux avec un affichage lisible
- faire des sélections de type base de données

In [6]:
import pandas as pd
from IPython.display import display, HTML

## Comptage des différentes formes BASE, PST, PART
On liste dans dLEMMA :
- les lemmes => key
- les formes de base (identiques au lemme normalement)
- les prétérits
- les participes passés

On compte les mêmes respectivement dans dBASE, dPST, dPART et le nombre de mots dans nb_word_corpus.

In [7]:
nb_word_corpus = 0
dPST={}
dPART={}
dBASE={}
dLEMMA={}
for x in range(len(list_cat)):
    cat = list_cat[x]
    raw = brown.raw(categories=cat)
    raw = raw.split()
    list_raw_word_tag = []
    for y in range(len(raw)):
        word_tag = raw[y]
        word_tag = word_tag.split("/")
        list_raw_word_tag.append(word_tag)
        
    for z in range(len(list_raw_word_tag)):
        word=raw[z].lower().split("/")[0]
        word_tag_check = list_raw_word_tag[z]
        check_isalpha = word_tag_check[0]
        if (check_isalpha != ".") and  (check_isalpha != ",") and (check_isalpha != "!") and (check_isalpha != "?") and (check_isalpha != ":") and (check_isalpha != "``") and (check_isalpha != "\'\'") and (check_isalpha != ";") and (check_isalpha != "-") and (check_isalpha != "--") and (check_isalpha != "(") and (check_isalpha != ")"):
            nb_word_corpus = nb_word_corpus + 1

        if (word_tag_check[1] == "vb"):
            lemma=lemmatizer.lemmatize(word,pos="v")
            if lemma not in dBASE: dBASE[lemma]=0
            dBASE[lemma]+=1
            if lemma not in dLEMMA:
                dLEMMA[lemma]={}
            if "BASE" not in dLEMMA[lemma]:
                dLEMMA[lemma]["BASE"]=word


            
        if (word_tag_check[1] == "vbd"):
            lemma=lemmatizer.lemmatize(word,pos="v")
            if lemma not in dPST: dPST[lemma]=0
            dPST[lemma]+=1
            if lemma not in dLEMMA:
                dLEMMA[lemma]={}
            if "PST" not in dLEMMA[lemma]:
                dLEMMA[lemma]["PST"]=word

        if (word_tag_check[1] == "vbn"):
            lemma=lemmatizer.lemmatize(word,pos="v")
            if lemma not in dPART: dPART[lemma]=0
            dPART[lemma]+=1
            if lemma not in dLEMMA:
                dLEMMA[lemma]={}
            if "PART" not in dLEMMA[lemma]:
                dLEMMA[lemma]["PART"]=word

print("Number of words in the corpus: ", nb_word_corpus)

Number of words in the corpus:  1013825


## Transformation dict=>DataFrame
On transforme les dictionnaires en tableaux Pandas.
- dfAllLemmas.T permet de transposer la matrice
- dfLemmas[["BASE","PST","PART"]] permet de fixer l'ordre des colonnes pour faciliter la lecture des tableaux

In [8]:
dfAllLemmas=pd.DataFrame(dLEMMA)
dfLemmas=dfAllLemmas.T
dfLemmas=dfLemmas[["BASE","PST","PART"]]
# dfLemmas[dfLemmas["BASE"]=="see"]

## Transformation dict=>Series
On transforme les dict de fréquence, *dBASE*, *dPST* & *dPART*, en séries Pandas pour constituer un tableau global *df* avec les formes et les occurrences.

In [9]:
dfBases=pd.Series(dBASE)
dfPreterites=pd.Series(dPST)
dfParticiples=pd.Series(dPART)

## Formation des triplets
on met toutes les informations dans *df* et ensuite on filtre.
- minElement donne le nombre d'occurrence minimal de chaque forme pour inclure un triplet
- limitTriplet est le filtre correspondant à minElement pour les trois formes
- dfTriplets ne garde que les triples complets (dropna) dont les occurrences sont toutes supérieures à minElement

In [10]:
df=pd.concat([dfLemmas,dfBases,dfPreterites, dfParticiples], axis=1)
dfTriplets=df.dropna().sort_values(0,ascending=False)
dfTriplets.columns="BASE PST PART oBASE oPST oPART".split(" ")
minElement=50
limitTriplet=(dfTriplets["oBASE"]>minElement) & (dfTriplets["oPST"]>minElement) & (dfTriplets["oPART"]>minElement)
dfTriplets=dfTriplets[limitTriplet]
display(dfTriplets)

,BASE,PST,PART,oBASE,oPST,oPART
make,make,made,made,791.0,466.0,654.0
get,get,got,got,742.0,338.0,156.0
know,know,knew,known,674.0,394.0,246.0
go,go,went,gone,613.0,507.0,194.0
take,take,took,taken,605.0,426.0,281.0
say,say,said,said,482.0,1748.0,213.0
think,think,thought,thought,433.0,340.0,74.0
come,come,came,come,431.0,618.0,191.0
find,find,found,found,398.0,268.0,267.0
give,give,gave,given,387.0,285.0,375.0


## Séparation en homographes/hétérographes
- les homographes ont le PST identique au PART
- les hétérographes ont le PST différent du PART

df.columns.name permet de donner un titre à un tableau *df*.

In [ ]:
dfHomographes=dfTriplets[dfTriplets["PST"]==dfTriplets["PART"]]
dfHeterographes=dfTriplets[dfTriplets["PST"]!=dfTriplets["PART"]]

In [13]:
dfHomographes.columns.name = 'Homographes'
display(dfHomographes)
dfHeterographes.columns.name = 'Hétérographes'
display(dfHeterographes)

Homographes,BASE,PST,PART,oBASE,oPST,oPART
make,make,made,made,791.0,466.0,654.0
get,get,got,got,742.0,338.0,156.0
say,say,said,said,482.0,1748.0,213.0
think,think,thought,thought,433.0,340.0,74.0
find,find,found,found,398.0,268.0,267.0
tell,tell,told,told,262.0,286.0,127.0
keep,keep,kept,kept,257.0,115.0,69.0
use,use,used,used,228.0,137.0,474.0
put,put,put,put,196.0,130.0,109.0
leave,leave,left,left,191.0,157.0,181.0


Hétérographes,BASE,PST,PART,oBASE,oPST,oPART
know,know,knew,known,674.0,394.0,246.0
go,go,went,gone,613.0,507.0,194.0
take,take,took,taken,605.0,426.0,281.0
come,come,came,come,431.0,618.0,191.0
give,give,gave,given,387.0,285.0,375.0
become,become,became,become,235.0,246.0,124.0
show,show,showed,shown,202.0,138.0,169.0
write,write,wrote,written,106.0,179.0,154.0
begin,begin,began,begun,84.0,312.0,51.0
break,break,broke,broken,65.0,66.0,60.0


In [14]:
# numerosKalaba=[4]
%store dfHeterographes dfHomographes

Stored 'dfHeterographes' (DataFrame)
Stored 'dfHomographes' (DataFrame)
